In [1]:
#@title Import required libraries
import os
import random
import torch
from torchvision import transforms
from music21 import converter, instrument, note, chord
import numpy as np
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [2]:
#@title Install and import mido and midi
!pip install mido --quiet
from mido import MidiFile
!pip install pretty_midi --quiet
import pretty_midi

In [3]:
#@title Connect the drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
#@title Set Dataset paths

train_data_dir = "/content/drive/MyDrive/train"
test_data_dir = "/content/drive/MyDrive/test"
val_data_dir = "/content/drive/MyDrive/dev"

train_composer_names = [composer for composer in os.listdir(train_data_dir) if '_' not in composer]
print('The training data set contains:', train_composer_names)
test_composer_names = [composer for composer in os.listdir(test_data_dir) if '_' not in composer]
print('The training data set contains:',test_composer_names)
val_composer_names = [composer for composer in os.listdir(val_data_dir) if '_' not in composer]
print('The training data set contains:',val_composer_names)

The training data set contains: ['schumann', 'chopin', 'mendelssohn', 'hummel', 'bach', 'byrd', 'bartok', 'mozart', 'handel']
The training data set contains: ['schumann', 'mozart', 'mendelssohn', 'hummel', 'handel', 'chopin', 'bach', 'byrd', 'bartok']
The training data set contains: ['schumann', 'mozart', 'bartok', 'byrd', 'handel', 'hummel', 'chopin', 'mendelssohn', 'bach']


In [5]:
#@title Load files # Limiting to train files only for now # change this after testing the accuracy ## Change here #1

from mido import MidiFile
def load_midi_files(data_dir, composer_names):
    midi_files = []
    for composer in composer_names:
        composer_dir = os.path.join(data_dir, composer)
        for file_name in os.listdir(composer_dir):
            if file_name.endswith(".mid") and "_" not in file_name:
                midi_path = os.path.join(composer_dir, file_name)
                midi = MidiFile(midi_path)
                midi_files.append((composer, midi))
    return midi_files

train_midi_files = load_midi_files(train_data_dir, train_composer_names)
print('Done loading train data\n')
#test_midi_files = load_midi_files(test_data_dir, test_composer_names)
#print('Done loading test data\n')
#val_midi_files = load_midi_files(val_data_dir, val_composer_names)
#print('Done validation data\n')

Done loading train data



In [11]:
#@title load the files # The above funtion repeats for Train, Test and Validation #### IGNORE this for now

#from mido import MidiFile
# Load MIDI files and create dataset
#midi_files = []
#for composer in train_composer_names:
#    composer_dir = os.path.join(train_data_dir, composer)
#    for file_name in os.listdir(composer_dir):
#        if file_name.endswith(".mid"):
#            midi_path = os.path.join(composer_dir, file_name)
#            midi = MidiFile(midi_path)
#            midi_files.append((composer, midi))


In [7]:
#@title Define MIDI feature extraction and preprocessing functions

# Extract notes
def extract_notes(midi_path):
    midi_data = pretty_midi.PrettyMIDI(midi_path)
    # Extract number of notes
    num_notes = len(midi_data.instruments[0].notes)
    return num_notes

# Extract tempo and time signature
def extract_tempo_and_time_signature(midi_path):
    midi_data = pretty_midi.PrettyMIDI(midi_path)
    # Extract tempo and time signature information
    tempo_changes = midi_data.get_tempo_changes()
    first_tempo = tempo_changes[0][0]
    time_signature_changes = midi_data.time_signature_changes
    if time_signature_changes:
        first_time_signature = time_signature_changes[0].numerator, time_signature_changes[0].denominator
    else:
        first_time_signature = (4, 4)  # Default time signature
    return first_tempo, first_time_signature

# Extract instrumentation
def extract_instrumentation(midi_path):
    midi_data = pretty_midi.PrettyMIDI(midi_path)
    # Extract number of unique instruments used
    instruments_used = len(midi_data.instruments)
    return instruments_used

# Extract dynamics
def extract_dynamics(midi_path):
    midi_data = pretty_midi.PrettyMIDI(midi_path)
    # Extract dynamics information (velocity ranges)
    velocities = [note.velocity for instrument in midi_data.instruments for note in instrument.notes]
    min_velocity = min(velocities)
    max_velocity = max(velocities)
    return min_velocity, max_velocity

# Extract melodic patterns
def extract_melodic_patterns(midi_path):
    midi_data = pretty_midi.PrettyMIDI(midi_path)
    # Extract melodic patterns (for example, intervals between consecutive notes)
    intervals = []
    for instrument in midi_data.instruments:
        sorted_notes = sorted(instrument.notes, key=lambda note: note.start)
        for i in range(1, len(sorted_notes)):
            interval = sorted_notes[i].pitch - sorted_notes[i-1].pitch
            intervals.append(interval)
    return np.array(intervals)

# Extract harmonics
def extract_harmonic_analysis(midi_path):
    midi_data = pretty_midi.PrettyMIDI(midi_path)
    # Extract harmonic analysis features (e.g., key changes)
    key_changes = len(midi_data.key_signature_changes)
    # Analyze harmonic information by calculating chord progressions
    chord_changes = 0
    for instrument in midi_data.instruments:
        notes = instrument.notes
        if len(notes) > 1:
            sorted_notes = sorted(notes, key=lambda note: note.start)
            for i in range(1, len(sorted_notes)):
                interval = sorted_notes[i].start - sorted_notes[i-1].start
                if interval > 0.5:  # A threshold to determine chord change
                    chord_changes += 1
    return key_changes, chord_changes

In [8]:
#@title Define the PyTorch Dataset
import torch
from torch.utils.data import Dataset
class MidiDataset(Dataset):
    def __init__(self, data):
        self.data = data
    def __len__(self):
        return len(self.data)
    def __getitem__(self, index):
        composer, midi = self.data[index]
        return composer, midi

In [8]:
#@title Dataloaders
from torch.utils.data import DataLoader

batch_size = 32
train_midi_dataset = MidiDataset(train_midi_files)
train_midi_loader = DataLoader(train_midi_dataset, batch_size=batch_size, shuffle=True)
test_midi_dataset = MidiDataset(test_midi_files)
test_midi_loader = DataLoader(test_midi_dataset, batch_size=batch_size, shuffle=True)
val_midi_dataset = MidiDataset(val_midi_files)
val_midi_loader = DataLoader(val_midi_dataset, batch_size=batch_size, shuffle=True)



In [ ]:
#@title Collect datasets

# Define label encoder
label_encoder = LabelEncoder()
label_encoder.fit(train_composer_names)
train_dataset = []
for composer in train_composer_names:
    composer_dir = os.path.join(train_data_dir, composer)
    for file_name in os.listdir(composer_dir):
        if file_name.endswith(".mid"):
            midi_path = os.path.join(composer_dir, file_name)
            # Extract features from MIDI file
            num_notes = extract_notes(midi_path)
            tempo, time_signature = extract_tempo_and_time_signature(midi_path)
     #       instrumentation = extract_instrumentation(midi_path)
     #       min_velocity, max_velocity = extract_dynamics(midi_path)
            melodic_patterns = extract_melodic_patterns(midi_path)
            key_changes, chord_changes = extract_harmonic_analysis(midi_path)
            # Combine features into a single feature vector
            features = [
                num_notes, tempo, time_signature,
                #instrumentation, min_velocity, max_velocity,
                melodic_patterns, key_changes, chord_changes
            ]
            composer_label = label_encoder.transform([composer])[0]
            train_dataset.append((features, composer_label))
# Convert the train_dataset to a NumPy array
train_dataset = np.array(train_dataset, dtype=object)

print('Done collecting Train Dataset\n')



In [ ]:
#@title Extract feature for Test and Validations sets
label_encoder = LabelEncoder()
label_encoder.fit(test_composer_names)

test_dataset = []
for composer in test_composer_names:
    composer_dir = os.path.join(test_data_dir, composer)
    for file_name in os.listdir(composer_dir):
        if file_name.endswith(".mid"):
            midi_path = os.path.join(composer_dir, file_name)
            # Extract features from MIDI file
            num_notes = extract_notes(midi_path)
            tempo, time_signature = extract_tempo_and_time_signature(midi_path)
            instrumentation = extract_instrumentation(midi_path)
            min_velocity, max_velocity = extract_dynamics(midi_path)
            melodic_patterns = extract_melodic_patterns(midi_path)
            key_changes, chord_changes = extract_harmonic_analysis(midi_path)

            # Combine features into a single feature vector
            features = [
                num_notes, tempo, time_signature,
                instrumentation, min_velocity, max_velocity,
                *melodic_patterns, key_changes, chord_changes
            ]

            composer_label = label_encoder.transform([composer])[0]
            test_dataset.append((features, composer_label))

# Convert the train_dataset to a NumPy array
test_dataset = np.array(test_dataset, dtype=object)

print('Done collecting Test Dataset\n')

# Define label encoder
label_encoder = LabelEncoder()
label_encoder.fit(val_composer_names)

val_dataset = []
for composer in val_composer_names:
    composer_dir = os.path.join(val_data_dir, composer)
    for file_name in os.listdir(composer_dir):
        if file_name.endswith(".mid"):
            midi_path = os.path.join(composer_dir, file_name)
            # Extract features from MIDI file
            num_notes = extract_notes(midi_path)
            tempo, time_signature = extract_tempo_and_time_signature(midi_path)
            instrumentation = extract_instrumentation(midi_path)
            min_velocity, max_velocity = extract_dynamics(midi_path)
            melodic_patterns = extract_melodic_patterns(midi_path)
            key_changes, chord_changes = extract_harmonic_analysis(midi_path)

            # Combine features into a single feature vector
            features = [
                num_notes, tempo, time_signature,
                instrumentation, min_velocity, max_velocity,
                *melodic_patterns, key_changes, chord_changes
            ]

            composer_label = label_encoder.transform([composer])[0]
            val_dataset.append((features, composer_label))

# Convert the train_dataset to a NumPy array
val_dataset = np.array(val_dataset, dtype=object)

print('Done collecting val Dataset\n')

In [21]:
#@title Check the resulting dataset features
print(train_dataset)
feature_vector = train_dataset[0][0]
num_features = len(feature_vector)
print("Number of train features:", num_features)
#feature_vector = test_dataset[0][0]
#num_features = len(feature_vector)
#print("Number of test features:", num_features)
#feature_vector = val_dataset[0][0]
#num_features = len(feature_vector)
#print("Number of val features:", num_features)

[[list([419, 0.0, (4, 4), array([ -7,   3,   5,  -1,  -8,   8,  12, -17,  12,  12,  -7,   0, -19,
           7,  -4,  24, -19,  -1,   0,  -8,  20, -17,  12,  12,  -7,   0,
         -12,  -7,   3,  24, -19,  -1,  -9,   4,   5,  12,  -5,  12,  -7,
         -23,  11,  -7,  19,   5, -16,  -1,  -8,  -4,  12,  12,  -5,  12,
          -7,   0, -12,  -4,  -3,  31, -23,  -1,  -8,   3,   5,  12,  -5,
          12,  -7,   0, -12,  -4,  -3,  27, -19,  -1,  -8,   3,  17, -12,
           7,  12,  -7,   0, -16,  -3,   7,  20, -19,  -1,  -9,   9,  -5,
          17,  -5,  12,  -7,   0, -12, -11,   4,  24, -16,  -1,  12, -24,
          12,  -8,  15,  12,  -7,  12,  -3, -40,   7,  29,  -5, -22,  -2,
          31,   5, -14,   2,   4, -28,  -7,  40, -24,  22, -24,  22,  -8,
           3, -24,  -7,   9,  -2,  -7,  17, -10,  12,  12,  -5,  -3,   5,
         -12,  10, -12,  10,  -8, -16,  19, -12,   2,  -2,   7,   9,  -4,
         -12,  -5,   2,  12, -14,  12,   7, -19,  16,  -6,  11,  -2, -31,
          29, 

In [22]:
#@title Create data loaders
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
#val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=False)
#test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

In [23]:
#@title LSTM Model

class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        _, (hidden, _) = self.lstm(x)
        output = self.fc(hidden[-1])
        return output


In [25]:

feature_vector = train_dataset[0][0]
num_features = len(feature_vector)

# Instantiate the LSTM model
input_size = num_features # Set the input size based on extracted features
hidden_size = 128
num_classes = len(train_composer_names)
lstm_model = LSTMModel(input_size, hidden_size, num_classes)

In [28]:

# Split dataset into train and validation sets
train_data, val_data = train_test_split(train_dataset, test_size=0.2, random_state=42) # This is for testing purposes only. This will be replaced with test folder data later.

# Create data loaders
batch_size = 32
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=False)

# 4. Training and Evaluation
learning_rate = 0.001
num_epochs = 10
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(lstm_model.parameters(), lr=learning_rate)

def train_model(model, train_loader, val_loader, num_epochs, criterion, optimizer):
    for epoch in range(num_epochs):
        model.train()
        for features, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(features)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

        model.eval()
        val_loss = 0.0
        correct = 0
        total = 0
        with torch.no_grad():
            for features, labels in val_loader:
                outputs = model(features)
                val_loss += criterion(outputs, labels).item()
                _, predicted = outputs.max(1)
                total += labels.size(0)
                correct += predicted.eq(labels).sum().item()

        print(f"Epoch [{epoch+1}/{num_epochs}] - Validation Accuracy: {(100*correct/total):.2f}%")


In [29]:

# Train the LSTM model
train_model(lstm_model, train_loader, val_loader, num_epochs, criterion, optimizer)

TypeError: ignored